In [1]:
import pandas as pd
import requests
import numpy as np
from datetime import datetime as dt
from bs4 import BeautifulSoup as BS
from time import sleep
from random import randint
import regex as re
from babel.numbers import format_currency

In [2]:
#bring in gdp excel worksheet
gdp = pd.read_excel('../data/gdp.xlsx',header = [3], nrows = 3219)
gdp.head()

,Unnamed: 0,2018,2019,2020,2021,2021.1,2019.1,2020.1,2021.2,2021.3
0,United States,18433773560,18857877001,18326935490,19410602997,--,2.3,-2.8,5.9,--
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Alabama,200372623,203432712,199880770,209979322,--,1.5,-1.7,5.1,--
3,Autauga,1600812,1543336,1525591,1502153,24,-3.6,-1.1,-1.5,59
4,Baldwin,6786933,7132205,7142139,7830237,7,5.1,0.1,9.6,9


In [3]:
#rename unamed column
gdp = gdp.rename(columns = {'Unnamed: 0':'County'})
gdp.head()

,County,2018,2019,2020,2021,2021.1,2019.1,2020.1,2021.2,2021.3
0,United States,18433773560,18857877001,18326935490,19410602997,--,2.3,-2.8,5.9,--
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Alabama,200372623,203432712,199880770,209979322,--,1.5,-1.7,5.1,--
3,Autauga,1600812,1543336,1525591,1502153,24,-3.6,-1.1,-1.5,59
4,Baldwin,6786933,7132205,7142139,7830237,7,5.1,0.1,9.6,9


In [4]:
#pull out columns for particular counties
counties = gdp.loc[(gdp['County']=='Davidson')|(gdp['County']=='Fulton')|(gdp['County']=='Travis')]
counties

,County,2018,2019,2020,2021,2021.1,2019.1,2020.1,2021.2,2021.3
146,Fulton,163612,164931,164097,169793,68,0.8,-0.5,3.5,46
470,Fulton,157806081,167699100,159661422,174408744,1,6.3,-4.8,9.2,19
652,Fulton,861479,878194,744358,784252,57,1.9,-15.2,5.4,56
752,Fulton,641131,628468,625570,643134,72,-2,-0.5,2.8,71
1067,Fulton,170372,184701,178063,196071,97,8.4,-3.6,10.1,5
1912,Fulton,1736341,1779519,1663916,1732174,49,2.5,-6.5,4.1,46
1987,Davidson,4257394,4332130,4157745,4462582,23,1.8,-4,7.3,34
2141,Fulton,1849054,1843788,1839920,1973548,51,-0.3,-0.2,7.3,23
2351,Fulton,646717,646211,550636,587124,64,-0.1,-14.8,6.6,6
2533,Davidson,63228320,65988731,63008917,71718395,1,4.4,-4.5,13.8,3


In [5]:
#locate proper fulton county
fulton = counties.iloc[1].to_frame().transpose().reset_index(drop = True)
fulton

,County,2018,2019,2020,2021,2021.1,2019.1,2020.1,2021.2,2021.3
0,Fulton,157806081,167699100,159661422,174408744,1,6.3,-4.8,9.2,19


In [6]:
#locate proper davidson county
davidson = counties.iloc[9].to_frame().transpose().reset_index(drop=True)
davidson

,County,2018,2019,2020,2021,2021.1,2019.1,2020.1,2021.2,2021.3
0,Davidson,63228320,65988731,63008917,71718395,1,4.4,-4.5,13.8,3


In [7]:
#locate proper travis county
travis = counties.iloc[10].to_frame().transpose().reset_index(drop=True)
travis

,County,2018,2019,2020,2021,2021.1,2019.1,2020.1,2021.2,2021.3
0,Travis,109328928,114194234,116192625,129639407,3,4.5,1.7,11.6,17


In [8]:
#add city column
fulton['city'] = ['Atlanta']
fulton

,County,2018,2019,2020,2021,2021.1,2019.1,2020.1,2021.2,2021.3,city
0,Fulton,157806081,167699100,159661422,174408744,1,6.3,-4.8,9.2,19,Atlanta


In [9]:
#add city column
davidson['city'] = ['Nashville']
davidson

,County,2018,2019,2020,2021,2021.1,2019.1,2020.1,2021.2,2021.3,city
0,Davidson,63228320,65988731,63008917,71718395,1,4.4,-4.5,13.8,3,Nashville


In [10]:
#add city column
travis['city'] = ['Austin']
travis

,County,2018,2019,2020,2021,2021.1,2019.1,2020.1,2021.2,2021.3,city
0,Travis,109328928,114194234,116192625,129639407,3,4.5,1.7,11.6,17,Austin


In [11]:
#concatenating all counties
counties_gdp = pd.concat([fulton,davidson,travis])
counties_gdp

,County,2018,2019,2020,2021,2021.1,2019.1,2020.1,2021.2,2021.3,city
0,Fulton,157806081,167699100,159661422,174408744,1,6.3,-4.8,9.2,19,Atlanta
0,Davidson,63228320,65988731,63008917,71718395,1,4.4,-4.5,13.8,3,Nashville
0,Travis,109328928,114194234,116192625,129639407,3,4.5,1.7,11.6,17,Austin


In [12]:
#setting columns
counties_gdp = counties_gdp.loc[:,['County','city',2018,2019, 2020,2021, '2019.1','2020.1', '2021.2']]
counties_gdp

,County,city,2018,2019,2020,2021,2019.1,2020.1,2021.2
0,Fulton,Atlanta,157806081,167699100,159661422,174408744,6.3,-4.8,9.2
0,Davidson,Nashville,63228320,65988731,63008917,71718395,4.4,-4.5,13.8
0,Travis,Austin,109328928,114194234,116192625,129639407,4.5,1.7,11.6


In [13]:
#gross gdp DF
gross = counties_gdp[['city',2018,2019, 2020,2021]]
gross

,city,2018,2019,2020,2021
0,Atlanta,157806081,167699100,159661422,174408744
0,Nashville,63228320,65988731,63008917,71718395
0,Austin,109328928,114194234,116192625,129639407


In [14]:
#percent change DF
perc_change = counties_gdp[['city', '2019.1','2020.1', '2021.2']]
perc_change

,city,2019.1,2020.1,2021.2
0,Atlanta,6.3,-4.8,9.2
0,Nashville,4.4,-4.5,13.8
0,Austin,4.5,1.7,11.6


In [15]:
#percent change DF renaming
perc_change = perc_change.rename(columns = {'2019.1':'2019','2020.1':'2020', '2021.2':'2021'})
perc_change

,city,2019,2020,2021
0,Atlanta,6.3,-4.8,9.2
0,Nashville,4.4,-4.5,13.8
0,Austin,4.5,1.7,11.6


In [16]:
#first melt for gdp value
gross = pd.melt(gross, id_vars = ['city'],var_name = 'year',value_name = 'gross_gdp')
gross

,city,year,gross_gdp
0,Atlanta,2018,157806081
1,Nashville,2018,63228320
2,Austin,2018,109328928
3,Atlanta,2019,167699100
4,Nashville,2019,65988731
5,Austin,2019,114194234
6,Atlanta,2020,159661422
7,Nashville,2020,63008917
8,Austin,2020,116192625
9,Atlanta,2021,174408744


In [17]:
#formatting gross gdp column to currency
gross['gross_gdp'] = gross['gross_gdp'].apply(lambda x: format_currency(x, currency="USD", locale="en_US"))
gross

,city,year,gross_gdp
0,Atlanta,2018,"$157,806,081.00"
1,Nashville,2018,"$63,228,320.00"
2,Austin,2018,"$109,328,928.00"
3,Atlanta,2019,"$167,699,100.00"
4,Nashville,2019,"$65,988,731.00"
5,Austin,2019,"$114,194,234.00"
6,Atlanta,2020,"$159,661,422.00"
7,Nashville,2020,"$63,008,917.00"
8,Austin,2020,"$116,192,625.00"
9,Atlanta,2021,"$174,408,744.00"


In [18]:
#first melt for gdp % change
perc_change = pd.melt(perc_change, id_vars = ['city'],var_name = 'year',value_name = 'gdp_perc_change')
perc_change

,city,year,gdp_perc_change
0,Atlanta,2019,6.3
1,Nashville,2019,4.4
2,Austin,2019,4.5
3,Atlanta,2020,-4.8
4,Nashville,2020,-4.5
5,Austin,2020,1.7
6,Atlanta,2021,9.2
7,Nashville,2021,13.8
8,Austin,2021,11.6


In [19]:
#save gross to csv
gross.to_csv('../data/cleaned/economic/gross_gdp.csv', index = False)

In [20]:
#save percent change to csv
perc_change.to_csv('../data/cleaned/economic/gdp_perc_change.csv', index = False)

In [21]:
#pull in price parity csv
price_parity = pd.read_csv('../data/price_parity.csv', nrows = 9)
price_parity

,GeoFips,GeoName,LineCode,Description,2017,2018,2019,2020
0,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",2,RPPs: Goods,99.204,98.922,97.115,96.721
1,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",3,RPPs: Services: Housing,99.579,99.698,101.030,100.827
2,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",4,RPPs: Services: Utilities,106.890,107.125,109.706,109.721
3,12420,"Austin-Round Rock-Georgetown, TX (Metropolitan...",2,RPPs: Goods,97.357,94.217,96.152,97.025
4,12420,"Austin-Round Rock-Georgetown, TX (Metropolitan...",3,RPPs: Services: Housing,125.256,122.474,121.398,121.094
5,12420,"Austin-Round Rock-Georgetown, TX (Metropolitan...",4,RPPs: Services: Utilities,108.044,109.500,111.557,111.593
6,34980,"Nashville-Davidson--Murfreesboro--Franklin, TN...",2,RPPs: Goods,96.975,92.079,94.455,95.283
7,34980,"Nashville-Davidson--Murfreesboro--Franklin, TN...",3,RPPs: Services: Housing,101.561,99.548,101.886,102.031
8,34980,"Nashville-Davidson--Murfreesboro--Franklin, TN...",4,RPPs: Services: Utilities,100.105,103.869,102.326,102.343


In [22]:
#splitting up description column values
price_parity[['rpp','desc','service']] = price_parity['Description'].str.split(': ', expand = True)
price_parity

,GeoFips,GeoName,LineCode,Description,2017,2018,2019,2020,rpp,desc,service
0,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",2,RPPs: Goods,99.204,98.922,97.115,96.721,RPPs,Goods,None
1,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",3,RPPs: Services: Housing,99.579,99.698,101.030,100.827,RPPs,Services,Housing
2,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",4,RPPs: Services: Utilities,106.890,107.125,109.706,109.721,RPPs,Services,Utilities
3,12420,"Austin-Round Rock-Georgetown, TX (Metropolitan...",2,RPPs: Goods,97.357,94.217,96.152,97.025,RPPs,Goods,None
4,12420,"Austin-Round Rock-Georgetown, TX (Metropolitan...",3,RPPs: Services: Housing,125.256,122.474,121.398,121.094,RPPs,Services,Housing
5,12420,"Austin-Round Rock-Georgetown, TX (Metropolitan...",4,RPPs: Services: Utilities,108.044,109.500,111.557,111.593,RPPs,Services,Utilities
6,34980,"Nashville-Davidson--Murfreesboro--Franklin, TN...",2,RPPs: Goods,96.975,92.079,94.455,95.283,RPPs,Goods,None
7,34980,"Nashville-Davidson--Murfreesboro--Franklin, TN...",3,RPPs: Services: Housing,101.561,99.548,101.886,102.031,RPPs,Services,Housing
8,34980,"Nashville-Davidson--Murfreesboro--Franklin, TN...",4,RPPs: Services: Utilities,100.105,103.869,102.326,102.343,RPPs,Services,Utilities


In [23]:
#assigning string type to column to deal with None values
price_parity['service'].astype(str)

0         None
1      Housing
2    Utilities
3         None
4      Housing
5    Utilities
6         None
7      Housing
8    Utilities
Name: service, dtype: object

In [24]:
#replacing None value with Goods
price_parity = price_parity.replace(to_replace ={ None: 'Goods'})
price_parity

,GeoFips,GeoName,LineCode,Description,2017,2018,2019,2020,rpp,desc,service
0,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",2,RPPs: Goods,99.204,98.922,97.115,96.721,RPPs,Goods,Goods
1,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",3,RPPs: Services: Housing,99.579,99.698,101.030,100.827,RPPs,Services,Housing
2,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",4,RPPs: Services: Utilities,106.890,107.125,109.706,109.721,RPPs,Services,Utilities
3,12420,"Austin-Round Rock-Georgetown, TX (Metropolitan...",2,RPPs: Goods,97.357,94.217,96.152,97.025,RPPs,Goods,Goods
4,12420,"Austin-Round Rock-Georgetown, TX (Metropolitan...",3,RPPs: Services: Housing,125.256,122.474,121.398,121.094,RPPs,Services,Housing
5,12420,"Austin-Round Rock-Georgetown, TX (Metropolitan...",4,RPPs: Services: Utilities,108.044,109.500,111.557,111.593,RPPs,Services,Utilities
6,34980,"Nashville-Davidson--Murfreesboro--Franklin, TN...",2,RPPs: Goods,96.975,92.079,94.455,95.283,RPPs,Goods,Goods
7,34980,"Nashville-Davidson--Murfreesboro--Franklin, TN...",3,RPPs: Services: Housing,101.561,99.548,101.886,102.031,RPPs,Services,Housing
8,34980,"Nashville-Davidson--Murfreesboro--Franklin, TN...",4,RPPs: Services: Utilities,100.105,103.869,102.326,102.343,RPPs,Services,Utilities


In [25]:
#splitting out city from Geoname value
price_parity[['city','1','2','3','4','5']] = price_parity['GeoName'].str.split('-',expand = True)
price_parity

,GeoFips,GeoName,LineCode,Description,2017,2018,2019,2020,rpp,desc,service,city,1,2,3,4,5
0,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",2,RPPs: Goods,99.204,98.922,97.115,96.721,RPPs,Goods,Goods,Atlanta,Sandy Springs,"Alpharetta, GA (Metropolitan Statistical Area)",None,None,None
1,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",3,RPPs: Services: Housing,99.579,99.698,101.030,100.827,RPPs,Services,Housing,Atlanta,Sandy Springs,"Alpharetta, GA (Metropolitan Statistical Area)",None,None,None
2,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",4,RPPs: Services: Utilities,106.890,107.125,109.706,109.721,RPPs,Services,Utilities,Atlanta,Sandy Springs,"Alpharetta, GA (Metropolitan Statistical Area)",None,None,None
3,12420,"Austin-Round Rock-Georgetown, TX (Metropolitan...",2,RPPs: Goods,97.357,94.217,96.152,97.025,RPPs,Goods,Goods,Austin,Round Rock,"Georgetown, TX (Metropolitan Statistical Area)",None,None,None
4,12420,"Austin-Round Rock-Georgetown, TX (Metropolitan...",3,RPPs: Services: Housing,125.256,122.474,121.398,121.094,RPPs,Services,Housing,Austin,Round Rock,"Georgetown, TX (Metropolitan Statistical Area)",None,None,None
5,12420,"Austin-Round Rock-Georgetown, TX (Metropolitan...",4,RPPs: Services: Utilities,108.044,109.500,111.557,111.593,RPPs,Services,Utilities,Austin,Round Rock,"Georgetown, TX (Metropolitan Statistical Area)",None,None,None
6,34980,"Nashville-Davidson--Murfreesboro--Franklin, TN...",2,RPPs: Goods,96.975,92.079,94.455,95.283,RPPs,Goods,Goods,Nashville,Davidson,,Murfreesboro,,"Franklin, TN (Metropolitan Statistical Area)"
7,34980,"Nashville-Davidson--Murfreesboro--Franklin, TN...",3,RPPs: Services: Housing,101.561,99.548,101.886,102.031,RPPs,Services,Housing,Nashville,Davidson,,Murfreesboro,,"Franklin, TN (Metropolitan Statistical Area)"
8,34980,"Nashville-Davidson--Murfreesboro--Franklin, TN...",4,RPPs: Services: Utilities,100.105,103.869,102.326,102.343,RPPs,Services,Utilities,Nashville,Davidson,,Murfreesboro,,"Franklin, TN (Metropolitan Statistical Area)"


In [26]:
#pulling out relevant columns 
price_parity = price_parity.loc[:,['city','service','2017','2018','2019','2020']]
price_parity 

,city,service,2017,2018,2019,2020
0,Atlanta,Goods,99.204,98.922,97.115,96.721
1,Atlanta,Housing,99.579,99.698,101.030,100.827
2,Atlanta,Utilities,106.890,107.125,109.706,109.721
3,Austin,Goods,97.357,94.217,96.152,97.025
4,Austin,Housing,125.256,122.474,121.398,121.094
5,Austin,Utilities,108.044,109.500,111.557,111.593
6,Nashville,Goods,96.975,92.079,94.455,95.283
7,Nashville,Housing,101.561,99.548,101.886,102.031
8,Nashville,Utilities,100.105,103.869,102.326,102.343


In [27]:
#saving price parity to csv 
price_parity.to_csv('../data/cleaned/economic/price_parity.csv', index = False)